# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
from openpyxl import load_workbook

# pd.set_option('display.max_rows', None)

# 连接数据库

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [3]:
# 该SQL语句用于查询满足特定条件的订单及其相关用户、商品和支付信息。
# 主要筛选条件包括：
# - 已还期数大于等于10期；
# - 无逾期记录（逾期期数和最大逾期天数均 <= 0）；
# - 总应收金额大于5000；
# - 订单不为续租订单；
# 同时关联了用户信息、订单详情、免押支付信息等多张表，用于构建完整的业务监控数据视图。


sql1='''SELECT tmu.true_name, tmu.id_card_num, tmu.mobile, tod.product_name, tprm.all_rental, tprm.total_receivable, tprm.total_received, tprm.buyout_balance_payment, 
tprm.purchase_amount, tprm.commodity_premium,tprm.platform_package,om.order_number, om.create_time, om.id, tprm.order_finish_date
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2
, om.has_actual, tprm.paid_periods, topay.total_freeze_fund_amount
from db_digua_business.t_postlease_receivables_monitoring tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id and om.del_flag = 0  
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
left join (SELECT t.order_id from db_rent.ya_merchant_order_stages t where t.is_relet>=1 GROUP BY 1) ymos 
on om.id = ymos.order_id 
-- where tprm.paid_periods >= 6 and tprm.paid_periods <= 10 and tprm.overdue_periods<=0 and tprm.max_overdue_days<=0 -- 6-10期已还且无逾期记录
where tprm.paid_periods >=10  and tprm.overdue_periods<=0 and tprm.max_overdue_days<=0
and tprm.total_receivable >5000  -- 总应收>5000
and ymos.order_id is null -- 非续租
'''

In [4]:
sql2='''SELECT DISTINCT b.mobile 
from db_digua_business.t_order a 
left join db_digua_business.t_member_user b on a.user_id = b.id
where a.create_time>= DATE_ADD(CURRENT_DATE,INTERVAL -90 day ) 
and a.create_time<= DATE_ADD(CURRENT_DATE,INTERVAL 0 day )               -- 近90天出库数据
and a.`status` in (2,3,4)
and b.mobile is not NULL
'''

In [5]:
# 近六个月下单
sql3 = '''
SELECT DISTINCT
om.order_number, om.user_mobile
FROM db_digua_business.t_order om 
left join db_digua_business.t_merchant tmer on tmer.id = om.merchant_id
WHERE om.create_time >= DATE_ADD(CURRENT_DATE, INTERVAL -5 MONTH)
and tmer.shop_type !=2
'''

In [6]:
# 黑名单
sql4 = '''
SELECT
id_card
FROM db_credit.blacklist b
WHERE b.end_time is NULL or  b.end_time>=DATE_ADD(CURRENT_DATE, INTERVAL -2 MONTH)
-- b.end_time>SYSDATE() 结束时间为空且大于当前时间，则为黑名单
'''

In [7]:
# 排除未完成且已付期数不足的订单
sql5 = '''
SELECT DISTINCT
om.order_number, om.user_mobile
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2
FROM db_digua_business.t_order om 
left join db_digua_business.t_merchant tmer on tmer.id = om.merchant_id
right join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
WHERE om.`status`!=8 and tmer.shop_type !=2 and tprm.max_overdue_days<=0 and tprm.overdue_periods<=0
and tprm.paid_periods < 10
'''

In [8]:
df = query(sql1)
df_pass90 = query(sql2)
df3 = query(sql3)
df4 = query(sql4)
df5 = query(sql5)
df.shape,df_pass90.shape

((3403, 19), (8749, 1))

# 处理信息

In [9]:
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")

# 获取已有的白名单数据

In [10]:
# 之前版本的白名单
df_white = pd.read_excel('F:/需求/白名单/ld白名单(1).xlsx', sheet_name='ld白名单')
df_white_list1 = df_white['id_card_num'].to_list()
df_white_list2 = df_white['mobile'].to_list()

In [11]:
# 每次导出的白名单
df_exits = pd.read_excel('F:/需求/白名单/白名单.xlsx')
df_exits_list = df_exits['身份证号'].to_list()

In [12]:
# 总重复的白名单
re_order_list = pd.read_excel('F:/需求/白名单/11月11总重复名单.xlsx')['手机号码'].astype(str).to_list()

# 获取日期

In [13]:
current_date = pd.Timestamp.now()
# 前一个月
before_date_1 = current_date - pd.DateOffset(months=1)
# 前两个月
before_date_2 = current_date - pd.DateOffset(months=2)
# 前六个月
before_date_6 = current_date - pd.DateOffset(months=6)
before_date_6

Timestamp('2024-08-17 18:49:22.034127')

# 剔除重复数据

In [14]:
# 剔除近90天出库的数据
df_w = df[~df['mobile'].isin(df_pass90.mobile.tolist())]

In [15]:
# 剔除ld白名单（1）的数据
df_tc = df[(~df['id_card_num'].isin(df_white_list1))]
df_tc = df[(~df['mobile'].isin(df_white_list2))]

In [16]:
# 剔除近半年下单的数据
mobile_list = df3['user_mobile'].to_list()
df_tc2 = df_tc[~df_tc['mobile'].isin(mobile_list)]

In [17]:
# 剔除黑名单的数据
df_tc2 = df_tc2[~df_tc2['id_card_num'].isin(df4['id_card'].to_list())]

In [18]:
# 排除近一个月的已买断订单
df_tc2['订单完成日期'] = pd.to_datetime(df_tc2[df_tc2['has_actual']==1]['order_finish_date']).dt.date
df_tc2list = df_tc2[pd.to_datetime(df_tc2['订单完成日期'])>=before_date_1.strftime('%Y-%m-%d')]['order_number'].to_list()
df_tc2 = df_tc2[~df_tc2['order_number'].isin(df_tc2list)]
df_tc2.rename(columns={'true_name':'姓名', 'id_card_num': '身份证号', 'mobile':'手机号'}, inplace=True)

In [19]:
# 排除非免押用户
df_tc2 = df_tc2[df_tc2['押金类型']!='部分免押']

In [20]:
# 排除已付期数不足且订单未完成未逾期的数据，
df_tc3_success_1 = df_tc2[['姓名', '身份证号', '手机号']]
df_tc3_success_1 = df_tc3_success_1[~df_tc3_success_1['手机号'].isin(df5['user_mobile'].to_list())]
df_tc3_success_1 = df_tc3_success_1[~df_tc3_success_1['身份证号'].isin(df_exits_list)]

In [21]:
# 排除总重复订单的数据
df_tc3_success_1 = df_tc3_success_1[~df_tc3_success_1.isin(re_order_list)]
df_tc3_success_1 = df_tc3_success_1[df_tc3_success_1['手机号'].notna()]

In [22]:
# 获取前300条数据
df_tc3_success_2 = df_tc3_success_1.drop_duplicates()[:300]

df_tc3_success_1

,姓名,身份证号,手机号
246,何健,522726199912259856,13232373191
879,柯佳俊,341021199703280013,18055910994
1077,战东明,232302200002031012,15246073323
1218,谢庚敖,510781200512031293,13198395717
1253,谭文彪,430424199010010872,18566743742
...,...,...,...
3359,冯军谕,530322200305022433,19531964689
3360,张轩,321323200507154918,17552641284
3362,王晓杰,130435199808061511,13345004588
3365,李治良,532301199407212910,18987804953


In [23]:
Today1 = datetime.datetime.now().strftime('%Y%m%d%H')
with pd.ExcelWriter(f'F:/需求/白名单/白名单_{Today1}.xlsx', engine='openpyxl') as writer:
    df_tc3_success_2.to_excel(writer, sheet_name='白名单', index=False)

# 在重复的名单中追加数据

In [24]:
# 将获取的白名单追加到需要排除的白名单下面

# 读取现有的 Excel 文件以获取最后一行的位置
File_Path = 'F:/需求/白名单/白名单.xlsx'
existing_df = pd.read_excel(File_Path)

# 获取现有数据的最后一行索引（行号）
last_row_index = existing_df.shape[0]+1

# 加载现有的工作簿
wb = load_workbook(filename=File_Path)
ws = wb.active

with pd.ExcelWriter(File_Path, engine='openpyxl', mode='a') as writer:
    
    writer.book = wb  # 使用已经加载的工作簿
    writer.sheets = {ws.title: ws for ws in wb.worksheets}  # 更新 writer 的 sheets 字典
    # 写入新数据，指定从最后一行之后开始
    df_tc3_success_2.to_excel(writer, sheet_name=ws.title, startrow=last_row_index, index=False, header=False)
    writer.save()

AttributeError: property 'book' of 'OpenpyxlWriter' object has no setter